T05: Koay Tze Min (A0188851N), Lai Yan Jean (A0190326J), Lee Jing Xuan (A0189467H), Mitchell Kwong (A0182695N), Valary Lim Wan Qian (A0190343L) 

# Loading Packages

In [ ]:
library(dplyr)
library(tidyverse)
library(Hmisc)
library(corrplot)
library(InformationValue)
library(MASS)
library(nnet)
library(naivebayes)
library(pracma)
library(psych)
library(CHAID)
library(partykit)
library(rpart)
library(rpart.plot)
library(caret)
library(randomForest)
library(ROSE)
library(e1071)

# Importing Dataset

In [ ]:
# Read card.csv dataset
data <- read.table('card.csv', sep=",", skip=2, header=FALSE)
header1 <- scan('card.csv', sep=",", nlines=1, what=character())
header2 <- scan('card.csv', sep=",", skip=1, nlines=1, what=character())

# Add headers to the data
header2[25] <- "DEFAULT"
colnames(data) <- header2

# Format columns
data <- mutate_at(data, c("SEX", "EDUCATION", "MARRIAGE", "PAY_0", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6", "DEFAULT"), as.factor)
str(data)

# Exploratory Data Analysis

In [ ]:
# Split the continuous and discrete data
continuous_data <- select(data, LIMIT_BAL, AGE, PAY_0, PAY_2, PAY_3, PAY_4, PAY_5, PAY_6,
                          BILL_AMT1, BILL_AMT2, BILL_AMT3, BILL_AMT4, BILL_AMT5, BILL_AMT6,
                          PAY_AMT1, PAY_AMT2, PAY_AMT3, PAY_AMT4, PAY_AMT5, PAY_AMT6, DEFAULT)
discrete_data <- select(data, SEX, EDUCATION, MARRIAGE, DEFAULT)

## Continuous Data

### Summarise Data

In [ ]:
summary(continuous_data[, 1:20])
continuous_data %>% describe()
# There are no missing values in the data.

### Hypothesis Test for Significance

In [ ]:
# Hypothesis Test for Significance
tmat <- data.frame(tstat = numeric(), pvalue = numeric())
for (i in 1:20) {
  result <- t.test(continuous_data[,i] ~ continuous_data$DEFAULT, var.equal = TRUE)
  tmat[i, 1] = result$statistic
  tmat[i, 2] = result$p.value
}
tmat <- cbind(names(continuous_data[,1:20]), tmat)
names(tmat) <- c("attribute", "t-statistic", "p-value")
tmat$significant <- ifelse(tmat$`p-value` < 0.05, 1, 0)

### Distribution Plots

In [ ]:
# Distribution: Limit Balance
boxplot(continuous_data$LIMIT_BAL, xlab = "Limit Balance", ylab = "NT Dollar", col = "darkslateblue")
ggplot(continuous_data, aes(x = LIMIT_BAL)) + geom_histogram(bins = 50, fill="darkslateblue")

In [ ]:
# Distribution: Age
boxplot(continuous_data$AGE, xlab = "Age", ylab = "Years", col = "darkslateblue")
ggplot(continuous_data, aes(x = AGE)) + geom_histogram(bins = 50, fill="darkslateblue")

In [ ]:
# Distribution: Payment Records
boxplot(continuous_data[,3:8], xlab = "Payment Records", col = "darkslateblue")

ggplot(continuous_data, aes(x = PAY_0)) + geom_histogram(bins = 10, fill="darkslateblue")
ggplot(continuous_data, aes(x = PAY_2)) + geom_histogram(bins = 10, fill="darkslateblue")
ggplot(continuous_data, aes(x = PAY_3)) + geom_histogram(bins = 10, fill="darkslateblue")
ggplot(continuous_data, aes(x = PAY_4)) + geom_histogram(bins = 10, fill="darkslateblue")
ggplot(continuous_data, aes(x = PAY_5)) + geom_histogram(bins = 10, fill="darkslateblue")
ggplot(continuous_data, aes(x = PAY_6)) + geom_histogram(bins = 10, fill="darkslateblue")

In [ ]:
# Distribution: Bill Amount
boxplot(continuous_data[,9:14], xlab = "Bill Amount", ylab = "NT Dollar", col = "darkslateblue")

ggplot(continuous_data, aes(x = BILL_AMT1)) + geom_histogram(bins = 30, fill="darkslateblue")
ggplot(continuous_data, aes(x = BILL_AMT2)) + geom_histogram(bins = 30, fill="darkslateblue")
ggplot(continuous_data, aes(x = BILL_AMT3)) + geom_histogram(bins = 30, fill="darkslateblue")
ggplot(continuous_data, aes(x = BILL_AMT4)) + geom_histogram(bins = 30, fill="darkslateblue")
ggplot(continuous_data, aes(x = BILL_AMT5)) + geom_histogram(bins = 30, fill="darkslateblue")
ggplot(continuous_data, aes(x = BILL_AMT6)) + geom_histogram(bins = 30, fill="darkslateblue")

In [ ]:
# Distribution: Payment Amount
boxplot(continuous_data[,15:20], xlab = "Payment Amount", ylab = "NT Dollar", col = "darkslateblue")

ggplot(continuous_data, aes(x = PAY_AMT1)) + geom_histogram(bins = 30, fill="darkslateblue")
ggplot(continuous_data, aes(x = PAY_AMT2)) + geom_histogram(bins = 30, fill="darkslateblue")
ggplot(continuous_data, aes(x = PAY_AMT3)) + geom_histogram(bins = 30, fill="darkslateblue")
ggplot(continuous_data, aes(x = PAY_AMT4)) + geom_histogram(bins = 30, fill="darkslateblue")
ggplot(continuous_data, aes(x = PAY_AMT5)) + geom_histogram(bins = 30, fill="darkslateblue")
ggplot(continuous_data, aes(x = PAY_AMT6)) + geom_histogram(bins = 30, fill="darkslateblue")

### Correlation Matrix

In [ ]:
# Correlation Matrix
corr.result <- rcorr(as.matrix(continuous_data)) # get mutual correlation
corrplot.mixed(corr.result$r, lower.col = "black", number.cex = .5, tl.cex = .45, tl.col = "black")

## Discrete Data

In [ ]:
# Converting to Factor Variable
data$SEX = as.factor(data$SEX)
data$EDUCATION = as.factor(data$EDUCATION)
data$MARRIAGE = as.factor(data$MARRIAGE)
data$DEFAULT = as.factor(data$DEFAULT)

discrete_data <- select(train.data, SEX, EDUCATION, MARRIAGE, DEFAULT)
View(discrete_data)

### Frequency Table

In [ ]:
sex_table <- table(discrete_data$SEX)
edu_table <- table(discrete_data$EDUCATION)
marriage_table <- table(discrete_data$MARRIAGE)
default_table <- table(discrete_data$DEFAULT)

### Bar Chart

In [ ]:
# Bar Chart: Sex
ylim <- c(0, 1.1*max(table(discrete_data$SEX)))
bar2 = barplot(table(discrete_data$SEX), names.arg = c("Male","Female"), main = "Sex", col = "darkslateblue", ylim = ylim)
text(x = bar2, y = table(discrete_data$SEX), label = table(discrete_data$SEX), pos = 3, cex = 0.8)

In [ ]:
# Bar Chart: Education
ylim <- c(0, 1.1*max(table(discrete_data$EDUCATION)))
bar3 = barplot(table(discrete_data$EDUCATION),main = "Education level", col = "darkslateblue",ylim = ylim)
text(x = bar3, y = table(discrete_data$EDUCATION), label = table(discrete_data$EDUCATION), pos = 3, cex = 0.8)

In [ ]:
# Bar Chart: Marriage
ylim <- c(0, 1.1*max(table(discrete_data$MARRIAGE)))
bar4 = barplot(table(discrete_data$MARRIAGE), main = "Marital Status", col = "darkslateblue", ylim = ylim)
text(x = bar4, y = table(discrete_data$MARRIAGE), label = table(discrete_data$MARRIAGE), pos = 3, cex = 0.8)

In [ ]:
# Bar Chart: Default (Target Variable)
ylim <- c(0, 1.1*max(table(discrete_data$DEFAULT)))
bar1 = barplot(table(discrete_data$DEFAULT), main = "Default", names.arg = c("No","Yes"), col = "darkslateblue",ylim = ylim)
text(x = bar1, y = table(discrete_data$DEFAULT), label = table(discrete_data$DEFAULT), pos = 3, cex = 0.8)

### Chi-Square Test

In [ ]:
chisq.test(discrete_data$SEX, discrete_data$DEFAULT)

In [ ]:
chisq.test(discrete_data$EDUCATION, discrete_data$DEFAULT)

In [ ]:
chisq.test(discrete_data$MARRIAGE, discrete_data$DEFAULT)

# Data Pre-Processing

### Splitting the Data

In [ ]:
# Make training and testing sets
set.seed(123)
n <- length(data$DEFAULT)
index <- 1:nrow(data)
testindex <- sample(index, trunc(2 * n) / 3)
test.data <- data[testindex, ]
train.data <- data[-testindex, ]

print(paste('shape(Xtr) = (', nrow(train.data), ',', ncol(train.data), ')'))
print(paste('shape(Xte) = (', nrow(test.data), ',', ncol(test.data), ')'))

In [ ]:
# Export data to csv
write.csv(train.data, paste(path, 'data', 'train.csv', sep='/'), row.names=FALSE)
write.csv(test.data, paste(path, 'data', 'test.csv', sep='/'), row.names=FALSE)

### Encoding Variables
The next few sections were coded in Python.

In [ ]:
# Import packages from Python
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, scale

In [ ]:
# Read data and initialise column names
df = pd.read_csv(f'{path}/data/card.csv', skiprows=1)

ID = ['ID']
continuous = ['LIMIT_BAL', 'AGE']
categories = ['SEX', 'EDUCATION', 'MARRIAGE']
paycols = ['PAY_'+i for i in '023456']
billcols = [col+i for col in ('BILL_AMT', 'PAY_AMT') for i in '123456']
target = ['default payment next month']

In [ ]:
# Defining code to encode
def encode(df):
  # work on a fresh copy
  cleaned = df.copy()

  # Categorical data
  cleaned['SEX_male'] = np.where(cleaned['SEX'] == 1, 1, 0)

  cleaned['EDUCATION_gradSch'] = np.where(cleaned['EDUCATION'] == 1, 1, 0)
  cleaned['EDUCATION_university'] = np.where(cleaned['EDUCATION'] == 2, 1, 0)
  cleaned['EDUCATION_highSch'] = np.where(cleaned['EDUCATION'] == 3, 1, 0)

  cleaned['MARRIAGE_married'] = np.where(cleaned['MARRIAGE'] == 1, 1, 0)
  cleaned['MARRIAGE_single'] = np.where(cleaned['MARRIAGE'] == 2, 1, 0)

  # Mixed data
  delay = [1, 2, 3, 4, 5, 6, 7, 8, 9]

  for col in paycols:
    cleaned[col+'_unknown0'] = np.where(cleaned[col] == 0, 1, 0)
    cleaned[col+'_unknown2'] = np.where(cleaned[col] == -2, 1, 0)
    cleaned[col+'_payDuly'] = np.where(cleaned[col] == -1, 1, 0)
    cleaned[col+'_value'] = cleaned[col].isin(delay) * cleaned[col]

  new_paycols = sum(([col+'_unknown0', col+'_unknown2', col+'_payDuly', col+'_value'] for col in paycols), [])

  orderedcols = [
      *ID,
      'LIMIT_BAL',
      'SEX_male', 
      'EDUCATION_gradSch', 'EDUCATION_university','EDUCATION_highSch',
      'MARRIAGE_married', 'MARRIAGE_single',
      'AGE',
      *new_paycols,
      *billcols,
      *target,
  ]

  colnames = [
      *ID,
      'LIMIT_BAL',
      'SEX_male', 
      'EDUCATION_gradSch', 'EDUCATION_university','EDUCATION_highSch',
      'MARRIAGE_married', 'MARRIAGE_single',
      'AGE',
      *new_paycols,
      *billcols,
      'DEFAULT',
  ]

  export = cleaned[orderedcols]
  export.columns = colnames

  return export

In [ ]:
# Encoding train set and exporting it
train = pd.read_csv(f'{path}/data/test.csv')
encode(train).to_csv(f'{path}/data/encoded_train.csv', index=False)

In [ ]:
# Encoding test set and exporting it
test = pd.read_csv(f'{path}/data/test.csv')
encode(train).to_csv(f'{path}/data/encoded_test.csv', index=False)

### Normalising Variables

In [ ]:
to_scale = continuous + billcols + [f'PAY_{i}_value' for i in '023456']

train = pd.read_csv(f'{path}/data/encoded_train.csv')
test = pd.read_csv(f'{path}/data/encoded_test.csv')

scaled_train = (train - train.mean()) / train.std()
scaled_test = (test - train.mean()) / train.std()

export_train = train.copy()
export_test = test.copy()

export_train[to_scale] = scaled_train[to_scale]
export_test[to_scale] = scaled_test[to_scale]

In [ ]:
# Exporting to csv
export_train.to_csv(f'{path}/data/card_train.csv', index=False)
export_test.to_csv(f'{path}/data/card_test.csv', index=False)

### Balancing Data by Oversampling

In [ ]:
skewed = pd.read_csv(f'{path}/data/card_train.csv')

counts = skewed.groupby('DEFAULT')['DEFAULT'].count()

print(counts)

In [ ]:
neg = skewed[skewed['DEFAULT']==0]
pos = skewed[skewed['DEFAULT']==1]

train_under = pd.concat([
    neg.sample(counts.min()),
    pos,
])

train_over = pd.concat([
    skewed,
    pos.sample(counts.max() - counts.min(), replace=True)
])

print(
    'After undersampling:',
    train_under.groupby('DEFAULT')['DEFAULT'].count(),
    '',
    'After oversampling:',
    train_over.groupby('DEFAULT')['DEFAULT'].count(),
    sep='\n',
)

In [ ]:
# Shuffle rows
train_under.sample(len(train_under), replace=False)
train_over.sample(len(train_over), replace=False)

train_under.to_csv(f'{path}/data/card_train_undersample.csv', index=False)
train_over.to_csv(f'{path}/data/card_train_oversample.csv', index=False)

# Models

In [ ]:
# Read oversampled train data
initial.train.data <- read.csv("card_train_oversample.csv")
# Remove the ID column
initial.train.data <- initial.train.data[, -1]

# Read test data
test.data <- read.csv("test_data.csv")
# Remove the ID column
test.data <- test.data[, -1]

## Logistic Regression

In [ ]:
# Read full train dataset that is unbalanced
full.train.data <- read.csv("card_train.csv")
# Remove the ID column
full.train.data <- full.train.data[,-1]
# Change decision variable to factor
full.train.data$DEFAULT = as.factor(full.train.data$DEFAULT)
View(full.train.data)

In [ ]:
# Splitting train-validation sets
folds <- cut(seq(1,nrow(full.train.data)), breaks = 10, labels = FALSE)

#### Full Logistic Regression (without Feature Selection)

In [ ]:
# Initialise f1 vector
f1 <- NULL

# Perform 10 fold cross validation
for (i in 1:10) {
  # Segment data by fold using the which() function
  val.indexes <- which(folds == i, arr.ind = TRUE)
  
  # Split train and test data
  val.data <- full.train.data[val.indexes, ]
  train.data <- full.train.data[-val.indexes, ]
  
  # Generate the model
  model = glm(DEFAULT~., data = train.data, family = binomial) 
  
  # Get optimal cut off
  lr.pred <- predict(model, newdata = train.data[,-45],type="response")
  optcut <- optimalCutoff(train.data$DEFAULT, lr.pred, optimiseFor="Both")
  
  # Predict on validation data using optimal cut off from train data 
  lr.pred.val <- predict(model, newdata = val.data[,-45],type="response")
  lr.binpred <- ifelse(lr.pred.val < optcut ,0,1)
  
  # Calculate f1 score
  recall = sensitivity(actuals = val.data$DEFAULT, predictedScores = lr.binpred, threshold = optcut)
  prec = precision(actuals = val.data$DEFAULT, predictedScores = lr.binpred, threshold = optcut)
  F1.score = 2 *(recall * prec)/(recall + prec)
  f1[i] <- F1.score
}

#### Akaike Information Criterion (AIC)

In [ ]:
# Initialise full model for backward feature selection
fullmodel = glm(DEFAULT ~ ., data = full.train.data, family = binomial) 

# Optimising model using AIC
lr.model <- stepAIC(fullmodel, direction = "backward", trace = FALSE)

# Initialise f1 vector
f1.aic <- NULL

# Creating formula
formula = as.formula(paste("DEFAULT ~ ", paste(row.names(as.matrix(coef(lr.model)))[-1], collapse = "+")))

# Perform 10 fold cross validation
for (i in 1:10) {
  # Segment data by fold using the which() function
  val.indexes <- which(folds == i, arr.ind = TRUE)
  
  # Split train and test data
  val.data <- full.train.data[val.indexes, ]
  train.data <- full.train.data[-val.indexes, ]
  
  # Generate the model
  model = glm(formula, data = train.data, family = binomial) 
  
  # Get optimal cut off
  lr.pred <- predict(model, newdata = train.data[,-45],type="response")
  optcut1 <- optimalCutoff(train.data$DEFAULT, lr.pred, optimiseFor="Both")
  
  # Predict on validation data using optimal cut off from train data 
  lr.pred.val <- predict(model, newdata = val.data[,-45],type="response")
  lr.binpred <- ifelse(lr.pred.val < optcut1 ,0,1)
  
  # Calculate f1 score
  recall = sensitivity(actuals = val.data$DEFAULT, predictedScores = lr.binpred, threshold = optcut1)
  prec = precision(actuals = val.data$DEFAULT, predictedScores = lr.binpred, threshold = optcut1)
  F1.score = 2 *(recall * prec)/(recall + prec)
  f1.aic[i] <- F1.score
}

#### Iterative Feature Selection (using p-Value)

In [ ]:
# Initialise full model for backward selection
fullmodel = glm(DEFAULT ~ ., data = full.train.data, family = binomial) 
model.p = fullmodel
formula.p = NULL

# Initialise the p-values from the model for iteration
p.values = as.matrix(summary(model.p)$coefficients[,4])
p.values = as.matrix(p.values[rownames(p.values) != "(Intercept)",])

# Iteratively remove variables with p-values > 0.05 until all p-values < 0.05
while (any(p.values > 0.05)){
  # Removing variable with the maximum p-value
  p.values = subset(p.values, p.values!=max(p.values))
  variables = row.names(p.values)
  variables
  
  # Creating formula
  formula.p = as.formula(paste("DEFAULT ~ ", paste(variables, collapse = "+")))
  
  # Create model
  model.p = glm(formula.p, data = full.train.data, family = binomial)
  
  # Retrieve and update variables and p-values for the next iteration
  p.values = as.matrix(summary(model.p)$coefficients[,4])
  p.values = as.matrix(p.values[rownames(p.values) != "(Intercept)",])
}

# Initialise F1 score
f1.p <- NULL

# Perform 10 fold cross validation
for (i in 1:10) {
  # Segment data by fold using the which() function
  val.indexes <- which(folds == i, arr.ind = TRUE)
  
  # Split train and test data
  val.data <- full.train.data[val.indexes, ]
  train.data <- full.train.data[-val.indexes, ]
  
  # Generate the model  
  model.p = glm(formula.p, data = train.data, family = binomial) 
  
  # Get optimal cut off
  lr.pred <- predict(model.p, newdata = train.data[,-45],type="response")
  optcut2 <- optimalCutoff(train.data$DEFAULT, lr.pred, optimiseFor="Both")
  
  # Predict on validation data using optimal cut off from train data 
  lr.pred.val <- predict(model.p, newdata = val.data[,-45],type="response")
  lr.binpred <- ifelse(lr.pred.val < optcut2 ,0,1)
  
  # Calculate f1 score
  recall = sensitivity(actuals = val.data$DEFAULT, predictedScores = lr.binpred, threshold = optcut2)
  prec = precision(actuals = val.data$DEFAULT, predictedScores = lr.binpred, threshold = optcut2)
  F1.score = 2 *(recall * prec)/(recall + prec)
  f1.p[i] <- F1.score
}

#### Comparing Different Logistic Regression Models

In [ ]:
# Create matrix consisting of the results of the cross validation (mean F1 score)
mat = as.matrix(c(mean(f1), mean(f1.aic),mean(f1.p)))
colnames(mat) = c("Mean F1 score")
rownames(mat) = c("Full model","AIC","Iterative")
# Best model found is AIC model

#### Building Final Model

In [ ]:
# Building overall regression model using best model from validation
pred <- predict(lr.model, newdata = full.train.data[,-45],type="response")
# Get optimal cut off
optcut.final <- optimalCutoff(full.train.data$DEFAULT, pred, optimiseFor="Both")
# Save Model
saveRDS(lr.model, file = "model_lr.rda")

## Decision Tree

#### Classification Tree
We first tried to build a basic classification tree model using the default parameters.

In [ ]:
# rpart (Recursive Partitioning) Algorithm
# Based on CART modelling which uses GINI index

# Grow tree
cart.tree1 <- rpart(`DEFAULT` ~., data=train.data[-1], method = 'class')
# Display the results
printcp(cart.tree1) 
# Detailed summary of splits
summary(cart.tree1) 
# Visualise cross-validation results
plotcp(cart.tree1) 

In [ ]:
# Plot Tree
plot(cart.tree1, main = "Classification Tree")
text(cart.tree1, use.n = TRUE, xpd = TRUE, cex = 0.8)

# Postscript Plot
post(cart.tree1, file = "tree.ps", title = "Classification Tree")

We performed evaluation on the train set using the initial model built, pruned the tree then performed an evaluation on the test set.

In [ ]:
# Evaluation - Train Set
train.pred1 <- predict(cart.tree1, type = "class")
train.cm <- table(pred=train.pred1, actual=train.data$DEFAULT)
train.cm.recall <- train.cm[2,2] / (train.cm[2,2] + train.cm[1,2])
train.cm.precision <- train.cm[2,2] / (train.cm[2,1] + train.cm[2,2])
train.cm.f1 <- (2 * train.cm.precision * train.cm.recall) / (train.cm.precision + train.cm.recall)

train.cm.f1 #0.6229132

In [ ]:
# Prune Tree
cart.tree1.pruned = prune(cart.tree1, cp = 0.0100)
# Choose the one that has the lowest xerror
plot(cart.tree1.pruned, main = "Classification Tree Pruned")
text(cart.tree1.pruned, cex = 0.9, xpd = TRUE)

In [ ]:
# Accuracy Analysis - Test Set
test.pred1 <- predict(cart.tree1.pruned, test.data[, c(1:45)], type = "class")
test.cm <- table(pred=test.pred1, actual=test.data$DEFAULT)
test.cm.recall <- test.cm[2,2] / (test.cm[2,2] + test.cm[1,2])
test.cm.precision <- test.cm[2,2] / (test.cm[2,1] + test.cm[2,2])
test.cm.f1 <- (2 * test.cm.precision * test.cm.recall) / (test.cm.precision + test.cm.recall)

test.cm.f1 #0.5132392

#### Model Validation
We tuned the model using different values of cp, using 10-fold cross validation.

In [ ]:
# Try different cp (complexity control parameter) to craft CART Tree
tryCP <- c(0.01, 0.001, 0.0001, 0.00001, 0.000001, 0.0000001, 0.00000001, 0.000000001)
# Create dataframe to store the result
result <- data.frame(matrix(ncol=3, nrow=8))
rCols <- c("cp", "val.accuracy", "val.f1")
colnames(result) <- rCols

# Create 10 equally size folds
initial.train.data <- train.data
folds <- cut(seq(1,nrow(initial.train.data)), breaks = 10, labels = FALSE)

index <- 1

for (i in tryCP) {
  # Initialise Accuracy and F1
  val.accuracy <- 0
  val.f1 <- 0
  
  # Perform 10 fold cross validation
  for (k in 1:10) {
    # Segment data by fold using the which() function
    val.indexes <- which(folds == k, arr.ind = TRUE)
    
    # Split train and test data
    val.data <- initial.train.data[val.indexes, ]
    train.data <- initial.train.data[-val.indexes, ]
    
    # Craft Decision Tree
    cart.tree <- rpart(`DEFAULT` ~., data = train.data[-1], method = 'class',
                       control = rpart.control(cp = i))
    
    # Prune Tree
    cart.tree.pruned = prune(cart.tree, 
                             cp = i)

    # Predict on Validation Set
    val.pred <- predict(cart.tree.pruned, val.data[, c(1:44)], type = "class")
    
    # Create confusion matrix
    CM <- table(pred = val.pred, actual = val.data$DEFAULT)
    
    # Metrics
    TP <- CM[2,2]
    TN <- CM[1,1]
    FP <- CM[2,1]
    FN <- CM[1,2]

    recall <- TP / (TP + FN)
    precision <- TP / (FP + TP)
    
    # Accuracy Analysis - Validation Set
    accuracy <- (TP + TN) / sum(CM)
    val.accuracy <- val.accuracy + accuracy
    
    # F1 Analysis - Validation Set
    f1 <- (2 * precision * recall) / (precision + recall)
    val.f1 <- val.f1 + f1
  }
  
  # Get average Accuracy
  val.accuracy <- val.accuracy / 10
  
  # Get average F1 score
  val.f1 <- val.f1 / 10

  # Add to results
  res <- c(i,
           val.accuracy,
           val.f1)
  
  result[index, ] <- res
  index <- index + 1
}

print(result)

#### Final Model Training
We built our final model based on the best cp value obtained from model validation.

In [ ]:
# Final Model Training
# grow tree
cart.tree.final <- rpart(`DEFAULT` ~., data=train.data[-1], method = 'class',
                         control = rpart.control(cp = 0.0001))
# Display the results
printcp(cart.tree.final) 

In [ ]:
# Summary Splits
# Detailed summary of splits
summary(cart.tree.final)

In [ ]:
# Plot Validation
# Visualise cross-validation results
plotcp(cart.tree.final) 

In [ ]:
# Plot Tree
plot(cart.tree.final, main = "Classification Tree")
text(cart.tree.final, use.n = TRUE, xpd = TRUE, cex = 0.8)

# Postscript Plot
post(cart.tree.final, file = "tree.ps", title = "Classification Tree")

In [ ]:
# Prune Tree
cart.tree.final.pruned = prune(cart.tree.final, cp = 0.0001)
# Plot Tree
plot(cart.tree.final.pruned, main = "Classification Tree Pruned")
text(cart.tree.final.pruned, cex = 0.9, xpd = TRUE)

In [ ]:
# Save Models
saveRDS(cart.tree.final, file = "model_CARTtree.rda")
saveRDS(cart.tree.final.pruned, file = "model_CARTtreePruned.rda")

### Random Forest

#### Building Initial Random Forest Model

In [ ]:
set.seed(123)
rf.model <- randomForest(`DEFAULT`~., data=train.data, importance=TRUE)

print(rf.model)
# mtry: Number of variables randomly sampled as candidates at each split.
# ntree: Number of trees to grow.

In [ ]:
# Basic Evaluation Function
evaluate <- function(prediction, decision) {
  table <- table(pred=prediction, actual=decision)
  print(table)
  
  #Metrics
  TP <- table[2,2]
  TN <- table[1,1]
  FP <- table[2,1]
  FN <- table[1,2]
  
  accuracy <- (TP + TN) / sum(table)
  precision <- TP / (TP + FP)
  recall <- TP / (TP + FN)
  F1 <- (2 * precision * recall) / (precision + recall)
  
  cat('- Accuracy', accuracy, '\n')
  cat('- Precision', precision, '\n')
  cat('- Recall', recall, '\n')
  cat('- F1 Score', F1, '\n')
}

In [ ]:
# Evaluating Train Data
train.data$rf.prediction <- predict(rf.model, train.data[, c(1:44)])
evaluate(train.data$rf.prediction, train.data$DEFAULT)

In [ ]:
# Evaluating Test Data
test.data$rf.prediction <- predict(rf.model, test.data[, c(1:44)])
evaluate(test.data$rf.prediction, test.data$DEFAULT)

In [ ]:
# Variable Importance
importance(rf.model)
varImpPlot(rf.model)

#### Model Tuning

In [ ]:
# Setting up results dataframe to store results
result <- data.frame(matrix(ncol=4, nrow=20))
rCols <- c("ntree", "mtry", "val.accuracy", "val.f1")
colnames(result) <- rCols

# Split train into train-validation (80-20)
initial.train.data <- train.data
set.seed(123)
n = length(initial.train.data$DEFAULT)
index <- 1:nrow(initial.train.data)
valindex <- sample(x = index, size = trunc(n * 0.2))
val.data <- initial.train.data[valindex,]
train.data <- initial.train.data[-valindex,]

index <- 1

In [ ]:
# Model Tuning
for (ntree in seq(100, 2000, 100)) {
  set.seed(123);
  rf.model.tune1 <- tuneRF(x = train.data[, c(1:44)], y = train.data$DEFAULT,
                           stepFactor = 1.5, improve = 1e-5, ntree=ntree)
  
  # Find optimal mtry
  optimal.mtry <- rf.model.tune1[which.min(rf.model.tune1[,"OOBError"]),"mtry"]
  
  # Build model
  set.seed(123)
  rf.model <- randomForest(`DEFAULT`~., data=train.data, importance=TRUE,
                           mtry = optimal.mtry, ntree = ntree)
  
  # Predict on Validation Set
  # Exclude decision column
  val.pred <- predict(rf.model, val.data[, c(1:44)])

  # Create confusion matrix
  CM <- table(pred = val.pred, actual = val.data$DEFAULT)
    
  # Metrics
  TP <- CM[2,2]
  TN <- CM[1,1]
  FP <- CM[2,1]
  FN <- CM[1,2]

  recall <- TP / (TP + FN)
  precision <- TP / (TP + FP)
    
  # Accuracy Analysis - Validation Set
  val.accuracy <- (TP + TN) / sum(CM)
    
  # F1 Analysis - Validation Set
  val.f1 <- (2 * precision * recall) / (precision + recall)

  # Add to results
  res <- c(ntree, optimal.mtry, val.accuracy, val.f1)
  result[index, ] <- res
  index <- index + 1
}

In [ ]:
print(result)
# Sort result in decreasing order based on F1, then accuracy
sorted <- result[order(-result$val.f1, -result$val.accuracy), ]
print(sorted)

#### Build Final Model

In [ ]:
# Build final model on entire train set
# Choose ntree = 1600, mtry = 28
rf.model.final <- randomForest(`DEFAULT`~., data=train.data, importance=TRUE, 
                               ntree = 1600, mtry = 28)

In [ ]:
# Evaluate final model on train set
train.pred <- predict(rf.model.final, train.data[, c(1:44)])
evaluate(train.pred, train.data$DEFAULT)

In [ ]:
# Evaluate final model on test set
test.pred <- predict(rf.model.final, test.data[, c(1:44)])
evaluate(test.pred, test.data$DEFAULT)

In [ ]:
# Save final model
saveRDS(rf.model.final, file = "model_rf.rda")

## Naive Bayes

#### Search Functions

In [ ]:
# Grid search
grid.search <- function(model.func, data, param.grid, ...) {
    # Initialize result table
    n.metrics <- length(list(...))
    metrics <- data.frame(matrix(nrow=0, ncol=2*n.metrics)) # x2 for train and validation
    
    # k-fold cross validation
    k <- 10
    data <- data %>% mutate(fold=cut(seq(1,nrow(data)), breaks = k, labels = FALSE))

    # Evaluate each parameter combination
    n.params <- nrow(param.grid)
    for (i in 1:n.params) {
        
        # Temp results table
        metrics.fold <- data.frame(matrix(nrow=0, ncol=2*n.metrics))

        # k fold cross validation
        for (j in 1:k) {
            # Split train and test data
            train.data <- data %>% filter(fold!=j)
            valid.data <- data %>% filter(fold==j)

            # Fit model
            params.data <- list(data=train.data)
            params.grid <- param.grid %>% slice(i) %>% flatten()
            params <- append(params.data, params.grid)
            model.fit <- do.call(model.func, params)

            # Model predictions
            train.actual <- train.data$DEFAULT
            train.pred <- predict(model.fit, type='prob')[,2]
            
            valid.actual <- valid.data$DEFAULT
            valid.pred <- predict(model.fit, newdata = select(valid.data, -DEFAULT), type='prob')[,2]
            
            # Evaluate metrics
            train.metrics <- list(...) %>% sapply((function(f) f(train.actual, train.pred)))
            valid.metrics <- list(...) %>% sapply((function(f) f(valid.actual, valid.pred)))
            metrics.fold <- rbind(metrics.fold, append(train.metrics, valid.metrics))
        }
        
        
        names(metrics.fold) <- c(paste('training', names(list(...)), sep='.'),
                                 paste('validation', names(list(...)), sep='.'))
        metrics <- rbind(metrics, metrics.fold %>% summarise_all(mean))
    }

    # Return metrics associated with each parameter combination
    names(metrics) <- c(paste('training', names(list(...)), sep='.'), 
                        paste('validation', names(list(...)), sep='.'))
  
    return(cbind(param.grid, metrics))
}

In [ ]:
# Re-build a model from the grid search
build.model <- function(model.func, data, params) {
    # Train on training set
    params.data <- list(data=data)
    params.grid <- params
    params <- append(params.data, params.grid)
    model.fit <- do.call(model.func, params)
    
    # return fitted model
    return(model.fit)
}

In [ ]:
# Metrics
getacc <- function(y, ypred) {
    # ypred is the predicted probability
    mean(y == ifelse(ypred > 0.5, 1, 0))
}


getAUC <- function(y, ypred) {
    M <- plotROC(y, ypred, returnSensitivityMat = TRUE)
    x <- M[,1]
    y <- M[,2]

    W <- x[2:length(x)] - x[1:(length(x)-1)]
    H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
    
    return(W %*% H)
}


getF1 <- function(y, ypred) {
    xtab <- table(factor(y, levels=c(1, 0)), 
                  factor(ypred > 0.5, levels=c(TRUE, FALSE)))
    
    precision <- xtab[1,1] / sum(xtab[,1])
    recall <- xtab[1,1] / sum(xtab[1,])

    return(2 * (precision * recall) / (precision + recall))
}

#### Search Parameters

In [ ]:
# Default args
prior.default = list(NULL)
laplace.default = list(0)
usekernel.default = list(FALSE)
usepoisson.default = list(FALSE)

In [ ]:
# Candidate args
xtab <- table(data.original$DEFAULT)
xtab <- xtab / sum(xtab)

prior = list(xtab, NULL)
laplace <- logseq(10e-5, 10e5, n=10)
usekernel = list(TRUE, FALSE)
usepoisson = list(TRUE, FALSE)

In [ ]:
# Naive Bayes model
func = function(...) {naive_bayes(DEFAULT~., ...)}

In [ ]:
# Minor data processing and feature selection using results from EDA
data.reduced <- data.balanced %>%
    select(LIMIT_BAL, PAY_AMT1, AGE, 
            PAY_0_value, PAY_0_payDuly, PAY_0_unknown0, PAY_0_unknown2,
            EDUCATION_gradSch, EDUCATION_university, EDUCATION_highSch, 
            MARRIAGE_married, MARRIAGE_single,
            SEX_male, DEFAULT) %>%
    mutate(DEFAULT=as.factor(DEFAULT))

#### Prior Probabilities
We tried to supply prior probabilities from the original dataset.

In [ ]:
results <-  grid.search(
    model.func = func,
    data = data.reduced,
    param.grid = expand.grid(
        prior = prior, 
        laplace = laplace.default, 
        usekernel = usekernel.default, 
        usepoisson = usepoisson.default
    ),

    # Metrics
    AUC = getAUC, 
    acc = getacc
)

# Arrange model params by training accuracy
results %>% arrange(desc(validation.acc))

#### Laplace
We search for a good laplace value over a logarithmic scale. This can be modified to iteratively narrow the range of the search.

In [ ]:
results <-  grid.search(
    model.func = func,
    data = data.reduced,
    param.grid = expand.grid(
        prior = prior.default,
        laplace = laplace, 
        usekernel = usekernel.default, 
        usepoisson = usepoisson.default
    ),

    # Metrics
    acc = getacc,
    f1 = getF1
)

# Arrange model params by training accuracy
results %>% arrange(desc(validation.f1))

#### Use Kernel
Most of our variables are not normally distributed so we expect the latter to perform better.

In [ ]:
results <-  grid.search(
    model.func = func,
    data = data.reduced,
    param.grid = expand.grid(
        prior = prior.default,
        laplace = laplace.default, 
        usekernel = usekernel, 
        usepoisson = usepoisson.default
    ),

    # Metrics
    AUC = getAUC, 
    acc = getacc
)

# Arrange model params by training accuracy
results %>% arrange(desc(validation.acc))

params.data <- list(data=train.data %>% mutate(DEFAULT=as.factor(DEFAULT)))
        params.grid <- param.grid %>% slice(i) %>% flatten()
        params <- append(params.data, params.grid)
        model.fit <- do.call(model.func, params)

#### Use Poisson

In [ ]:
results <-  grid.search(
    model.func = func,
    data = data.reduced,
    param.grid = expand.grid(
        prior = prior.default,
        laplace = laplace.default, 
        usekernel = usekernel.default, 
        usepoisson = usepoisson
    ),

    # Metrics
    AUC = getAUC, 
    acc = getacc
)

# Arrange model params by training accuracy
results %>% arrange(desc(validation.acc))

#### Grid Search for Best Parameters

In [ ]:
results <- grid.search(
    model.func = func,
    data = data.reduced,
    param.grid = expand.grid(
        prior = prior,
        laplace = laplace, 
        usekernel = usekernel, 
        usepoisson = usepoisson
    ),

    # Metrics
    acc = getacc,
    f1 = getF1
)

#### Validation Scores

In [ ]:
# Accuracy
acc.top10 <- results %>%
  arrange(desc(validation.acc)) %>%
  head(10)

options(width=150)
print('best params by validation accuracy')
print.data.frame(acc.top10)

acc.topParams <- acc.top10 %>% slice(1) %>% flatten()
acc.topModel <- build.model(func, data.reduced, acc.topParams)

summary(acc.topModel)
plot(acc.topModel)

# Save Model
file = paste(path, 'models', 'model_nb_acc.rda', sep='/')
saveRDS(acc.topModel, file = file)

In [ ]:
# F1 Score
f1.top10 <- results %>%
  arrange(desc(validation.f1)) %>%
  head(10)

options(width=150)
print('best params by validation F1 score')
print(f1.top10)

f1.topParams <- f1.top10 %>% slice(1) %>% flatten()
f1.topModel <- build.model(func, data.reduced, f1.topParams)

summary(f1.topModel)
plot(f1.topModel)

# Save Model
file = paste(path, 'models', 'model_nb_f1.rda', sep='/')
saveRDS(f1.topModel, file = file)

In [ ]:
print('Means across all models')
results %>% 
    select(training.acc, training.f1, validation.acc, validation.f1) %>%
    replace_na(list(validation.f1=0, training.f1=0)) %>%
    summarise_all(mean)

## Support Vector Machine

In [ ]:
# Load packages
library(ROSE)
library(e1071)

# Load data
balanced <- read.table("card_train_oversample.csv", sep = ",", header = T)
bal.d <- balanced %>% select(-c(ID)) # Remove ID columns

# Evaluation Metrics
F1 <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, newdata=data, type='class')
  
  xtab <- table(factor(y, levels=c(1, 0)), 
                factor(y.pred, levels=c(1, 0)))
  
  precision <- xtab[1,1] / sum(xtab[,1])
  recall <- xtab[1,1] / sum(xtab[1,])
  
  return(2 * (precision * recall) / (precision + recall))
}

Accuracy <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, newdata = data, type = "class")
  
  xtab <- table(factor(y, levels=c(1, 0)), 
                factor(y.pred, levels=c(1, 0)))
  accuracy <- (xtab[1,1] + xtab[2,2]) / length(y)
  
  return (accuracy)
}

# Function to create and test models
cvFunction <- function(fold, kernel, cost, gamma) {
  training_fold <- bal.d[-fold, ]
  val_fold <- bal.d[fold, ]
  classifier <- svm(
    DEFAULT ~.,
    data = training_fold,
    type = "C-classification",
    kernel = kernel,
    cost = cost,
    gamma = gamma
  )
  
  accuracy <- Accuracy(classifier, val_fold)
  f1score <- F1(classifier, val_fold)
  return(list("accuracy" = accuracy, "f1" = f1score))
}

#### Test Kernel Types

In [ ]:
results.type <- data.frame(Accuracy = numeric(), F1 = numeric())
kernelTypes <- c("linear", "radial", "polynomial", "sigmoid")
for (type in kernelTypes) {
  folds <- cut(seq(1,nrow(bal.d)), breaks = 10, labels = FALSE)
  
  accuracy <- 0
  f1 <- 0
  
  for (i in 1:10) {
    # Segment data by fold using the which() function
    val.indexes <- which(folds == i, arr.ind = TRUE)
    
    # Put it into the cvFunction
    res <- cvFunction(val.indexes, type, 0.1, 0.5)
    
    # Store each 
    accuracy <- accuracy + res$accuracy
    f1 <- f1 + res$f1
  }
  
  # Get mean
  meanAcc <- accuracy / 10
  meanF1 <- f1 / 10
  
  results.type <- rbind(results.type, c(meanAcc, meanF1))
}
results.type <- cbind(kernelTypes, results.type)
names(results.type) <- c("Kernel", "Accuracy", "F1")
write.csv(results.type, "SVM_results_type.csv")
results.type

#### Test Cost

In [ ]:
# Test cost
results.cost <- data.frame(Cost = numeric(), Accuracy = numeric(), F1 = numeric())
for (cost in 10^(-2:2)) {
  folds <- cut(seq(1,nrow(bal.d)), breaks = 10, labels = FALSE)
  
  accuracy <- 0
  f1 <- 0
  
  for (i in 1:10) {
    # Segment data by fold using the which() function
    val.indexes <- which(folds == i, arr.ind = TRUE)
    
    # Put it into the cvFunction
    res <- cvFunction(val.indexes, "radial", cost, 0.5)
    
    # Store each 
    accuracy <- accuracy + res$accuracy
    f1 <- f1 + res$f1
  }
  
  # Get mean
  meanAcc <- accuracy / 10
  meanF1 <- f1 / 10
  
  results.cost <- rbind(results.cost, c(cost, meanAcc, meanF1))
}
names(results.cost) <- c("Cost", "Accuracy", "F1")
write.csv(results,cost, "SVM_results_cost.csv")

#### Test Gamma

In [ ]:
results.gamma <- data.frame(Gamma = numeric(), Accuracy = numeric(), F1 = numeric())
for (gamma in 2^(-2:2)) {
  folds <- cut(seq(1,nrow(bal.d)), breaks = 10, labels = FALSE)
  
  accuracy <- 0
  f1 <- 0
  
  for (i in 1:10) {
    # Segment data by fold using the which() function
    val.indexes <- which(folds == i, arr.ind = TRUE)
    
    # Put it into the cvFunction
    res <- cvFunction(val.indexes, "radial", 0.1, gamma)
    
    # Store each 
    accuracy <- accuracy + res$accuracy
    f1 <- f1 + res$f1
  }
  
  # Get mean
  meanAcc <- accuracy / 10
  meanF1 <- f1 / 10
  
  results.gamma <- rbind(results.gamma, c(meanAcc, meanF1))
}
names(results.gamma) <- c("Gamma", "Accuracy", "F1")
write.csv(results.gamma, "SVM_results_gamma.csv")

#### Grid Search on Parameters

In [ ]:
# Grid search on parameteres
gs <- list(type = c("polynomial"), 
           cost = c(10, 20, 30, 40, 50, 60, 70, 80, 90, 100),
           gamma = c(0, 0.25, 0.5)) %>% 
  cross_df()

# Store results
results.gs <- data.frame(Cost = numeric(), Gamma = numeric(), Accuracy = numeric(), F1 = numeric())

for (i in 1:nrow(gs)) {
  i.cost <- gs$cost[i]
  i.gamma <- gs$gamma[i]
  i.type <- gs$type[i]
  print(paste("cost:", i.cost, ", gamma:", i.gamma))
  
  folds <- cut(seq(1,nrow(bal.d)), breaks = 10, labels = FALSE)
  
  accuracy <- 0
  f1 <- 0
  
  for (i in 1:10) {
    print(paste("cost:", i.cost, ", gamma:", i.gamma, ", iteration:", i))
    # Segment data by fold using the which() function
    val.indexes <- which(folds == i, arr.ind = TRUE)
    
    # Put it into the cvFunction
    res <- cvFunction(val.indexes, i.type, i.cost, i.gamma)
    
    # Store each 
    accuracy <- accuracy + res$accuracy
    f1 <- f1 + res$f1
  }
  
  # Get mean
  meanAcc <- accuracy / 10
  meanF1 <- f1 / 10
  
  print(paste("cost:", i.cost, ", gamma:", i.gamma, ", mean accuracy:", meanAcc, ", mean F1:", meanF1))
  results.gs <- rbind(results.gs, c(i.cost, i.gamma, meanAcc, meanF1))
}

# View results
names(results.gs) <- c("Cost", "Gamma", "Accuracy", "F1")
results.gs <- cbind(Kernel = "polynomial", results.gs)
write.csv(results.gs, "SVM_results.csv")
results.gs <- results.gs[order(-results.gs$F1),]
head(results.gs)

#### Final Model Training

In [ ]:
# Train the finalised model on the full train dataset 
set.seed(123)
model.svm <- svm(
  DEFAULT ~.,
  data = bal.d,
  type = "C-classification",
  kernel = "polynomial",
  cost = 60,
  gamma = 0.25
)

## Save the model (for future reference)
saveRDS(model.svm, file = "model_svm.rda")

## To load the model, uncomment the following line
model.svm <- readRDS("model_svm.rda")

## Neural Network

In [ ]:
# F1 Score
F1 <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, newdata=data, type='class')
  
  xtab <- table(factor(y, levels=c(1, 0)), 
                factor(y.pred, levels=c(1, 0)))
  
  precision <- xtab[1,1] / sum(xtab[,1])
  recall <- xtab[1,1] / sum(xtab[1,])
  
  return(2 * (precision * recall) / (precision + recall))
}

In [ ]:
# Set formula
formula <- as.formula(paste("train.class ~ ", paste(labels[2:45], collapse = "+")))
formula

In [ ]:
# Function to compute RMSE
RMSE.calc <- function(model, val.data) {
  mr <- model$residuals
  rmse_squared = 0
  n <- nrow(val.data)
  for (i in 1:n) {
    rmse_squared = rmse_squared + (mr[i, 1])**2
  }
  rmse_squared = rmse_squared / (n - 1)
  rmse = sqrt(rmse_squared)
  rmse
}

#### Test for Different Number of Hidden Layers

In [ ]:
# Create 10 equally size folds
folds <- cut(seq(1,nrow(initial.train.data)), breaks = 10, labels = FALSE)

In [ ]:
# Test for different number of hidden layers (size)
results.size <- data.frame(Size = numeric(), RMSE = numeric(), F1 = numeric())
size <- 1
while (size <= 21) {
  # Initialise RMSE and F1
  rmse <- 0
  f1 <- 0
  
  # Perform 10 fold cross validation
  for (i in 1:10) {
    # Segment data by fold using the which() function
    val.indexes <- which(folds == i, arr.ind = TRUE)
    
    # Split train and test data
    val.data <- initial.train.data[val.indexes, ]
    train.data <- initial.train.data[-val.indexes, ]
    
    train.class <- train.data[, 46]
    
    # Set seed and train model
    set.seed(123)
    model.nn <- nnet(formula, data = train.data, size = size, maxit = 1000, entropy = TRUE)
    
    # Calculate RMSE and Accuracy 
    rmse <- rmse + RMSE.calc(model.nn, val.data)
    f1 <- f1 + F1(model.nn, val.data)
  }
  
  # Get average RMSE and Accuracy
  rmse <- rmse / 10
  f1 <- f1 / 10
  
  # Add to results
  results.size <- rbind(results.size, c(size, rmse, f1))
  
  size = size + 5
}

In [ ]:
# View results
names(results.size) <- c("Size", "RMSE", "F1")
results.size

#### Test for Different Decays

In [ ]:
# Test for different decays
# Set default size = 10
results.decay <- data.frame(Decay = numeric(), RMSE = numeric(), F1 = numeric())
decay <- 0
while (decay <= 1) {
  # Initialise RMSE and F1
  rmse <- 0
  f1 <- 0
  
  # Perform 10 fold cross validation
  for (i in 1:10) {
    # Segment data by fold using the which() function
    val.indexes <- which(folds == i, arr.ind = TRUE)
    
    # Split train and test data
    val.data <- initial.train.data[val.indexes, ]
    train.data <- initial.train.data[-val.indexes, ]
    
    train.class <- train.data[, 46]
    
    # Set seed and train model
    set.seed(123)
    model.nn <- nnet(formula, data = train.data, size = 10, maxit = 1000, decay = decay, entropy = TRUE)
    
    # Calculate RMSE and Accuracy 
    rmse <- rmse + RMSE.calc(model.nn, val.data)
    f1 <- f1 + F1(model.nn, val.data)
  }
  
  # Get average RMSE and Accuracy
  rmse <- rmse / 10
  f1 <- f1 / 10
  
  # Add to results
  results.decay <- rbind(results.decay, c(decay, rmse, f1))
  
  decay = decay + 0.2
}

In [ ]:
# View results
names(results.decay) <- c("Decay", "RMSE", "F1")
results.decay

In [ ]:
# Plot the results
ggplot(results.decay, aes(x = Decay, y = Value)) + 
  geom_line(aes(y = RMSE), color = "darkred") + 
  geom_line(aes(y = Accuracy), color="steelblue", linetype="twodash")

#### Grid Search

In [ ]:
# Grid Search
results.gs <- data.frame(Size = numeric(), Decay = numeric(), F1 = numeric())

In [ ]:
# Define a named list of parameter values
# Intervals for decay were decreased from 0.1 during the testing phase to 0.025 increments to more closely
# tune the decay.
gs <- list(size = c(16, 17, 18, 19, 20, 21),
           decay = c(0.0, 0.025, 0.05, 0.075, 0.1, 0.125, 0.15, 0.175, 0.2)) %>% 
  cross_df() # Convert to data frame grid

for (i in 1:nrow(gs)) {
  i.size <- gs$size[i]
  i.decay <- gs$decay[i]
  
  # Initialise F1 score
  f1 <- 0
  
  # Perform 10 fold cross validation
  for (i in 1:10) {
    # Segment data by fold using the which() function
    val.indexes <- which(folds == i, arr.ind = TRUE)
    
    # Split train and test data
    val.data <- initial.train.data[val.indexes, ]
    train.data <- initial.train.data[-val.indexes, ]
    
    train.class <- train.data[, 46]
    
    # Set seed and train model
    set.seed(123)
    model.nn <- nnet(formula, data = train.data, size = i.size, maxit = 1000, decay = i.decay, entropy = TRUE)
    
    # Calculate F1
    f1 <- f1 + F1(model.nn, val.data)
  }
  
  f1 <- f1 / 10
  
  # Add to results
  results.gs <- rbind(results.gs, c(i.size, i.decay, f1))
}

In [ ]:
# View results
names(results.gs) <- c("Size", "Decay", "F1")
results.gs <- results.gs[order(-results.gs$F1),]
head(results.gs)

#### Final Model Training

In [ ]:
# Train the finalised model on the full train dataset 
# (size = 20, decay = 0, entropy = TRUE, maxit = 1000)
set.seed(123)
train.class <- initial.train.data$DEFAULT
model.nn <- nnet(formula, data = initial.train.data, size = 20, decay = 0, maxit = 1000, entropy = TRUE)

In [ ]:
## Save the model (for future reference)
saveRDS(model.nn, file = "model_nn.rda")

# Model Evaluation

## Logistic Regression

In [ ]:
# Load Logistic Regression Model
lr.model <- readRDS("model_lr.rda")

In [ ]:
# Evaluation Functions for Logistic Regression
acc <- function(model, data, optcut) {
  y = data$DEFAULT
  y.pred = predict(model, newdata=data, type='response')
  y.pred <- ifelse(y.pred < optcut ,0,1)
  
  return(mean(y == y.pred))
}


AUC <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, newdata=data, type='response')
  
  M <- plotROC(y, y.pred, returnSensitivityMat = TRUE)
  x <- M[,1]
  y <- M[,2]
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  return(W %*% H)
}


F1 <- function(model, data, optcut) {
  y = data$DEFAULT
  y.pred = predict(model, newdata=data, type='response')
  y.pred <- ifelse(y.pred < optcut ,0,1)
  
  xtab <- table(factor(y, levels=c(1, 0)), 
                factor(y.pred, levels=c(1, 0)))
  
  precision <- xtab[1,1] / sum(xtab[,1])
  recall <- xtab[1,1] / sum(xtab[1,])
  
  return(2 * (precision * recall) / (precision + recall))
}


gain <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, data, type = 'response')
  
  # Combine the actual and pred
  df <- as.data.frame(cbind(y, y.pred))
  names(df) <- c("actual", "pred")
  
  # Order the prediction scores in descending order
  df <- df[order(-df$pred),]
  
  # Split into deciles
  df <- df %>% mutate(quantile = ntile(pred, 10))
  
  # Get total no. of positive instances
  total.positive = nrow(data[data$DEFAULT == 1,])
  
  # Calculate gain
  df.gains <- data.frame(Percentile = numeric(), Gain = numeric())
  df.gains <- rbind(df.gains, c(0, 0))
  perc = 0.1
  for (i in 1:10) {
    n <- sum(df[df$quantile == (11-i),]$actual) # number of positive instances
    g <- n / total.positive
    df.gains <- rbind(df.gains, c(perc, g))
    perc <- perc + 0.1
  }
  names(df.gains) <- c("Percentile", "Gain")
  
  # Calculate cumulative gain
  df.gains$CumGain <- cumsum(df.gains$Gain)
  
  # Calculate and return area under cumulative gain chart
  x <- df.gains$Percentile
  y <- df.gains$CumGain
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  # Plot graph
  plot(df.gains$Percentile,df.gains$CumGain,type="l",xlab="Percentile",ylab="Gain", main = "Cumulative Gain Chart")
  
  return(W %*% H)
}


lift <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, data, type = 'response')
  
  # Combine the actual and pred
  df <- as.data.frame(cbind(y, y.pred))
  names(df) <- c("actual", "pred")
  
  # Order the prediction scores in descending order
  df <- df[order(-df$pred),]
  
  # Split into deciles
  df <- df %>% mutate(quantile = ntile(pred, 10))
  
  # Get total no. of positive instances
  perc.positive = nrow(data[data$DEFAULT == 1,]) / nrow(data)
  
  # Calculate lift
  df.lift <- data.frame(Percentile = numeric(), Lift = numeric())
  perc = 0.1
  for (i in 1:10) {
    n <- sum(df[df$quantile == (11-i),]$actual) # number of positive instances
    p <- n / nrow(df[df$quantile == (11-i),])
    l <- p / perc.positive
    df.lift <- rbind(df.lift, c(perc, l))
    perc <- perc + 0.1
  }
  names(df.lift) <- c("Percentile", "Lift")
  
  # Calculate cumulative gain
  df.lift$CumLift <- cumsum(df.lift$Lift)
  
  # Calculate and return area under cumulative gain chart
  x <- df.lift$Percentile
  y <- df.lift$Lift
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  # Plot graph
  plot(df.lift$Percentile, df.lift$Lift,type="l",xlab="Percentile",ylab="Lift", main = "Lift Chart")
  
  return(W %*% H)
}

In [ ]:
# Evaluating Test Set
acc(lr.model, test.data, optcut.final)
F1(lr.model, test.data, optcut.final)
AUC(lr.model, test.data)
gain(lr.model, test.data)
lift(lr.model, test.data)

## Decision Tree

In [ ]:
# Evaluation Functions for BOTH Decision Tree and Random Forest
acc <- function(model, data) {
    y = data$DEFAULT
    y.pred = predict(model, newdata=data, type='class')
    # y.pred = predict(model, newdata=data)

    return(mean(y == y.pred))
}


AUC <- function(model, data) {
    y = data$DEFAULT
    y.pred = predict(model, newdata=data, type='prob')[,2]
    
    M <- plotROC(y, y.pred, returnSensitivityMat = TRUE)
    x <- M[,1]
    y <- M[,2]

    W <- x[2:length(x)] - x[1:(length(x)-1)]
    H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
    
    return(W %*% H)
}


F1 <- function(model, data) {
    y = data$DEFAULT
    y.pred = predict(model, newdata=data, type='class')

    xtab <- table(factor(y, levels=c(1, 0)), 
                  factor(y.pred, levels=c(1, 0)))
    
    precision <- xtab[1,1] / sum(xtab[,1])
    recall <- xtab[1,1] / sum(xtab[1,])

    return(2 * (precision * recall) / (precision + recall))
}


gain <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, data, type = 'prob')[,2]
  
  # Combine the actual and pred
  df <- as.data.frame(cbind(y, y.pred))
  names(df) <- c("actual", "pred")
  df$actual <- ifelse(df$actual == 1, 0, 1)
  
  # Order the prediction scores in descending order
  df <- df[order(-df$pred),]
  
  # Split into deciles
  df <- df %>% mutate(quantile = ntile(pred, 10))
  
  # Get total no. of positive instances
  total.positive = nrow(data[data$DEFAULT == 1,])
  
  # Calculate gain
  df.gains <- data.frame(Percentile = numeric(), Gain = numeric())
  df.gains <- rbind(df.gains, c(0, 0))
  perc = 0.1
  for (i in 1:10) {
    n <- sum(df[df$quantile == (11-i),]$actual) # number of positive instances
    g <- n / total.positive
    df.gains <- rbind(df.gains, c(perc, g))
    perc <- perc + 0.1
  }
  names(df.gains) <- c("Percentile", "Gain")
  
  # Calculate cumulative gain
  df.gains$CumGain <- cumsum(df.gains$Gain)
  
  # Calculate and return area under cumulative gain chart
  x <- df.gains$Percentile
  y <- df.gains$CumGain
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  # Plot graph
  plot(df.gains$Percentile,df.gains$CumGain,type="l",xlab="Percentile",ylab="Gain", main = "Cumulative Gain Chart")
  
  return(W %*% H)
}


lift <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, data, type = 'prob')[,2]
  
  # Combine the actual and pred
  df <- as.data.frame(cbind(y, y.pred))
  names(df) <- c("actual", "pred")
  df$actual <- ifelse(df$actual == 1, 0, 1)
  
  # Order the prediction scores in descending order
  df <- df[order(-df$pred),]
  
  # Split into deciles
  df <- df %>% mutate(quantile = ntile(pred, 10))
  
  # Get total no. of positive instances
  perc.positive = nrow(data[data$DEFAULT == 1,]) / nrow(data)
  
  # Calculate lift
  df.lift <- data.frame(Percentile = numeric(), Lift = numeric())
  perc = 0.1
  for (i in 1:10) {
    n <- sum(df[df$quantile == (11-i),]$actual) # number of positive instances
    p <- n / nrow(df[df$quantile == (11-i),])
    l <- p / perc.positive
    df.lift <- rbind(df.lift, c(perc, l))
    perc <- perc + 0.1
  }
  names(df.lift) <- c("Percentile", "Lift")
  
  # Calculate cumulative gain
  df.lift$CumLift <- cumsum(df.lift$Lift)
  
  # Calculate and return area under cumulative gain chart
  x <- df.lift$Percentile
  y <- df.lift$Lift
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  # Plot graph
  plot(df.lift$Percentile, df.lift$Lift,type="l",xlab="Percentile",ylab="Lift", main = "Lift Chart")
  
  return(W %*% H)
}

In [ ]:
# Evaluate Test Set
acc(model.CART.pruned, test.data)
F1(model.CART.pruned, test.data)
lift(model.CART.pruned, test.data)
gain(model.CART.pruned, test.data)
AUC(model.CART.pruned, test.data)

## Random Forest

In [ ]:
# Evaluation Function Same as Decision Tree
# Evaluate Test Set
acc(model.rf, test.data)
F1(model.rf, test.data)
lift(model.rf, test.data)
gain(model.rf, test.data)
AUC(model.rf, test.data)

## Naive Bayes

In [ ]:
# Evaluation Functions for Naive Bayes
acc <- function(model, data) {
    y = data$DEFAULT
    y.pred = predict(model, newdata=data, type='class')
    # y.pred = predict(model, newdata=data)

    return(mean(y == y.pred))
}


AUC <- function(model, data) {
    y = data$DEFAULT
    y.pred = predict(model, newdata=data, type='prob')[,2]
    
    M <- plotROC(y, y.pred, returnSensitivityMat = TRUE)
    x <- M[,1]
    y <- M[,2]

    W <- x[2:length(x)] - x[1:(length(x)-1)]
    H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
    
    return(W %*% H)
}


F1 <- function(model, data) {
    y = data$DEFAULT
    y.pred = predict(model, newdata=data, type='class')
    # y.pred = predict(model, newdata=data)

    xtab <- table(factor(y, levels=c(1, 0)), 
                  factor(y.pred, levels=c(1, 0)))
    
    precision <- xtab[1,1] / sum(xtab[,1])
    recall <- xtab[1,1] / sum(xtab[1,])

    return(2 * (precision * recall) / (precision + recall))
}


gain <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, data, type = 'prob')[,2]
  
  # Combine the actual and pred
  df <- as.data.frame(cbind(y, y.pred))
  names(df) <- c("actual", "pred")
  
  # Order the prediction scores in descending order
  df <- df[order(-df$pred),]
  
  # Split into deciles
  df <- df %>% mutate(quantile = ntile(pred, 10))
  
  # Get total no. of positive instances
  total.positive = nrow(data[data$DEFAULT == 1,])
  
  # Calculate gain
  df.gains <- data.frame(Percentile = numeric(), Gain = numeric())
  df.gains <- rbind(df.gains, c(0, 0))
  perc = 0.1
  for (i in 1:10) {
    n <- sum(df[df$quantile == (11-i),]$actual) # number of positive instances
    g <- n / total.positive
    df.gains <- rbind(df.gains, c(perc, g))
    perc <- perc + 0.1
  }
  names(df.gains) <- c("Percentile", "Gain")
  
  # Calculate cumulative gain
  df.gains$CumGain <- cumsum(df.gains$Gain)
  
  # Calculate and return area under cumulative gain chart
  x <- df.gains$Percentile
  y <- df.gains$CumGain
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  # Plot graph
  plot(df.gains$Percentile,df.gains$CumGain,type="l",xlab="Percentile",ylab="Gain", main = "Cumulative Gain Chart")
  
  return(W %*% H)
}


lift <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, data, type = 'prob')[,2]
  
  # Combine the actual and pred
  df <- as.data.frame(cbind(y, y.pred))
  names(df) <- c("actual", "pred")
  
  # Order the prediction scores in descending order
  df <- df[order(-df$pred),]
  
  # Split into deciles
  df <- df %>% mutate(quantile = ntile(pred, 10))
  
  # Get total no. of positive instances
  perc.positive = nrow(data[data$DEFAULT == 1,]) / nrow(data)
  
  # Calculate lift
  df.lift <- data.frame(Percentile = numeric(), Lift = numeric())
  perc = 0.1
  for (i in 1:10) {
    n <- sum(df[df$quantile == (11-i),]$actual) # number of positive instances
    p <- n / nrow(df[df$quantile == (11-i),])
    l <- p / perc.positive
    df.lift <- rbind(df.lift, c(perc, l))
    perc <- perc + 0.1
  }
  names(df.lift) <- c("Percentile", "Lift")
  
  # Calculate cumulative gain
  df.lift$CumLift <- cumsum(df.lift$Lift)
  
  # Calculate and return area under cumulative gain chart
  x <- df.lift$Percentile
  y <- df.lift$Lift
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  # Plot graph
  plot(df.lift$Percentile, df.lift$Lift,type="l",xlab="Percentile",ylab="Lift", main = "Lift Chart")
  
  return(W %*% H)
}

In [ ]:
evaluate.models <- function(models, data, metrics) {
    result <- data.frame(matrix(nrow=0, ncol=length(metrics)))
    
    for (model.name in names(models)) {
        model <- models[[model.name]]
        model.metrics <- metrics %>% sapply((function(f) f(model, data)))
        result <- rbind(result, model.metrics)
    }

    result <- cbind(names(models), result)
    names(result) <- append('model', names(metrics))
    return(result)
}

In [ ]:
# Evaluating Test Set
data.test <- read.csv(paste(path, 'data', 'card_test.csv', sep='/')) %>% as_tibble()

models <- list(
    naive.bayes.acc = readRDS(paste(path, 'models', 'model_nb_acc.rda', sep='/')),
    naive.bayes.f1 = readRDS(paste(path, 'models', 'model_nb_f1.rda', sep='/')),
    naive.bayes.auc = readRDS(paste(path, 'models', 'model_nb_AUC.rda', sep='/'))
)

metrics <- list(
    accuracy = acc,
    f1 = F1,
    AUROC = AUC,
    lift = lift,
    gain = gain
)

evaluate.models(models = models, data = data.test, metrics = metrics)

## Support Vector Machine

In [ ]:
# Evaluation Function for SVM
acc <- function(model, data) {
    y = as.factor(data$DEFAULT)
    y.pred = predict(model, newdata=data)

    return(mean(y == y.pred))
}

AUC <- function(model, data) {
    y = data$DEFAULT
    y.pred = attr(predict(model, data, probability=TRUE), 'probabilities')[,1]
  
    M <- plotROC(y, y.pred, returnSensitivityMat = TRUE)
    x <- M[,1]
    y <- M[,2]

    W <- x[2:length(x)] - x[1:(length(x)-1)]
    H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
    
    return(W %*% H)
}

F1 <- function(model, data) {
    y = as.factor(data$DEFAULT)
    y.pred = predict(model, newdata=data)
    # y.pred = predict(model, newdata=data)

    xtab <- table(factor(y, levels=c(1, 0)), 
                  factor(y.pred, levels=c(1, 0)))
    
    precision <- xtab[1,1] / sum(xtab[,1])
    recall <- xtab[1,1] / sum(xtab[1,])

    return(2 * (precision * recall) / (precision + recall))
}

gain <- function(model, data) {
  y = data$DEFAULT
  y.pred = attr(predict(model, data, probability=TRUE), 'probabilities')[,1]
  
  # Combine the actual and pred
  df <- as.data.frame(cbind(y, y.pred))
  names(df) <- c("actual", "pred")
  
  # Order the prediction scores in descending order
  df <- df[order(-df$pred),]
  
  # Split into deciles
  df <- df %>% mutate(quantile = ntile(pred, 10))
  
  # Get total no. of positive instances
  total.positive = nrow(data[data$DEFAULT == 1,])
  
  # Calculate gain
  df.gains <- data.frame(Percentile = numeric(), Gain = numeric())
  df.gains <- rbind(df.gains, c(0, 0))
  perc = 0.1
  for (i in 1:10) {
    n <- sum(df[df$quantile == (11-i),]$actual) # number of positive instances
    g <- n / total.positive
    df.gains <- rbind(df.gains, c(perc, g))
    perc <- perc + 0.1
  }
  names(df.gains) <- c("Percentile", "Gain")
  
  # Calculate cumulative gain
  df.gains$CumGain <- cumsum(df.gains$Gain)
  
  # Calculate and return area under cumulative gain chart
  x <- df.gains$Percentile
  y <- df.gains$CumGain
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  # Plot graph
  plot(df.gains$Percentile,df.gains$CumGain,type="l",xlab="Percentile",ylab="Gain", main = "Cumulative Gain Chart")
  
  return(W %*% H)
}

lift <- function(model, data) {
  y = data$DEFAULT
  y.pred = attr(predict(model, data, probability=TRUE), 'probabilities')[,1]
  
  # Combine the actual and pred
  df <- as.data.frame(cbind(y, y.pred))
  names(df) <- c("actual", "pred")
  
  # Order the prediction scores in descending order
  df <- df[order(-df$pred),]
  
  # Split into deciles
  df <- df %>% mutate(quantile = ntile(pred, 10))
  
  # Get total no. of positive instances
  perc.positive = nrow(data[data$DEFAULT == 1,]) / nrow(data)
  
  # Calculate lift
  df.lift <- data.frame(Percentile = numeric(), Lift = numeric())
  perc = 0.1
  for (i in 1:10) {
    n <- sum(df[df$quantile == (11-i),]$actual) # number of positive instances
    p <- n / nrow(df[df$quantile == (11-i),])
    l <- p / perc.positive
    df.lift <- rbind(df.lift, c(perc, l))
    perc <- perc + 0.1
  }
  names(df.lift) <- c("Percentile", "Lift")
  
  # Calculate cumulative gain
  df.lift$CumLift <- cumsum(df.lift$Lift)
  
  # Calculate and return area under cumulative gain chart
  x <- df.lift$Percentile
  y <- df.lift$Lift
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  # Plot graph
  plot(df.lift$Percentile, df.lift$Lift,type="l",xlab="Percentile",ylab="Lift", main = "Lift Chart")
  
  return(W %*% H)
}

evaluate.models <- function(models, data, metrics) {
    result <- data.frame(matrix(nrow=0, ncol=length(metrics)))
    
    for (model.name in names(models)) {
        model <- models[[model.name]]
        model.metrics <- metrics %>% sapply((function(f) f(model, data)))
        result <- rbind(result, model.metrics)
    }

    result <- cbind(names(models), result)
    names(result) <- append('model', names(metrics))
    return(result)
}

In [ ]:
# Evaluating Test Set
data.test <- read.csv(paste('data', 'card_test.csv', sep='/')) %>% as_tibble()

models <- list(
    naive.bayes.acc = readRDS(paste(path, 'models', 'model_svm.rda', sep='/'))
)

metrics <- list(
    accuracy = acc,
    f1 = F1,
    AUROC = AUC,
    lift = lift,
    gain = gain
)

evaluate.models(models = models, data = data.test, metrics = metrics)

## Neural Network

In [ ]:
# Evaluation Functions for Neural Network
acc <- function(model, data) {
    y = data$DEFAULT
    y.pred = predict(model, newdata=data, type='class')
    return(mean(y == y.pred))
}


AUC <- function(model, data) {
    y = data$DEFAULT
    y.pred = predict(model, newdata=data, type='raw')
    
    M <- plotROC(y, y.pred, returnSensitivityMat = TRUE)
    x <- M[,1]
    y <- M[,2]

    W <- x[2:length(x)] - x[1:(length(x)-1)]
    H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
    
    return(W %*% H)
}


F1 <- function(model, data) {
    y = data$DEFAULT
    y.pred = predict(model, newdata=data, type='class')

    xtab <- table(factor(y, levels=c(1, 0)), 
                  factor(y.pred, levels=c(1, 0)))
    
    precision <- xtab[1,1] / sum(xtab[,1])
    recall <- xtab[1,1] / sum(xtab[1,])

    return(2 * (precision * recall) / (precision + recall))
}


gain <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, data, type = 'raw')
  
  # Combine the actual and pred
  df <- as.data.frame(cbind(y, y.pred))
  names(df) <- c("actual", "pred")
  
  # Order the prediction scores in descending order
  df <- df[order(-df$pred),]
  
  # Split into deciles
  df <- df %>% mutate(quantile = ntile(pred, 10))
  
  # Get total no. of positive instances
  total.positive = nrow(data[data$DEFAULT == 1,])
  
  # Calculate gain
  df.gains <- data.frame(Percentile = numeric(), Gain = numeric())
  df.gains <- rbind(df.gains, c(0, 0))
  perc = 0.1
  for (i in 1:10) {
    n <- sum(df[df$quantile == (11-i),]$actual) # number of positive instances
    g <- n / total.positive
    df.gains <- rbind(df.gains, c(perc, g))
    perc <- perc + 0.1
  }
  names(df.gains) <- c("Percentile", "Gain")
  
  # Calculate cumulative gain
  df.gains$CumGain <- cumsum(df.gains$Gain)
  
  # Calculate and return area under cumulative gain chart
  x <- df.gains$Percentile
  y <- df.gains$CumGain
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  # Plot graph
  plot(df.gains$Percentile,df.gains$CumGain,type="l",xlab="Percentile",ylab="Gain", main = "Cumulative Gain Chart")
  
  return(W %*% H)
}


lift <- function(model, data) {
  y = data$DEFAULT
  y.pred = predict(model, data, type = 'raw')
  
  # Combine the actual and pred
  df <- as.data.frame(cbind(y, y.pred))
  names(df) <- c("actual", "pred")
  
  # Order the prediction scores in descending order
  df <- df[order(-df$pred),]
  
  # Split into deciles
  df <- df %>% mutate(quantile = ntile(pred, 10))
  
  # Get total no. of positive instances
  perc.positive = nrow(data[data$DEFAULT == 1,]) / nrow(data)
  
  # Calculate lift
  df.lift <- data.frame(Percentile = numeric(), Lift = numeric())
  perc = 0.1
  for (i in 1:10) {
    n <- sum(df[df$quantile == (11-i),]$actual) # number of positive instances
    p <- n / nrow(df[df$quantile == (11-i),])
    l <- p / perc.positive
    df.lift <- rbind(df.lift, c(perc, l))
    perc <- perc + 0.1
  }
  names(df.lift) <- c("Percentile", "Lift")
  
  # Calculate cumulative gain
  df.lift$CumLift <- cumsum(df.lift$Lift)
  
  # Calculate and return area under cumulative gain chart
  x <- df.lift$Percentile
  y <- df.lift$Lift
  
  W <- x[2:length(x)] - x[1:(length(x)-1)]
  H <- (y[2:length(x)] + y[1:(length(x)-1)]) / 2
  
  # Plot graph
  plot(df.lift$Percentile, df.lift$Lift,type="l",xlab="Percentile",ylab="Lift", main = "Lift Chart")
  
  return(W %*% H)
}

In [ ]:
# Evaluating Test Set
data.test <- read.csv(paste(path, 'data', 'card_test.csv', sep='/')) %>% as_tibble()

models <- list(
    neural.network = readRDS(paste(path, 'models', 'model_nn.rda', sep='/'))
)

metrics <- list(
    accuracy = acc,
    f1 = F1,
    AUROC = AUC,
    lift = lift,
    gain = gain
)

evaluate.models(models = models, data = data.test, metrics = metrics)